In [154]:
import numpy as np
import pandas as pd
import re
import nltk
import math
from IPython.display import display, HTML


In [96]:
# 玉山資料
yushan = pd.read_csv('./data/job.csv')
yushan['job_title'] = yushan['job_title'].apply(lambda x: str(x).strip())
display(yushan.head())

# 主計處資料
gov = pd.read_excel('./data/psdnquery1.xlsx')
display(gov.head())

,job_title,occupation,quota_origin,quota_now,quota_origin_elec,quota_now_elec
0,維護工程師,21.0,1000000.0,500000.0,NaN,NaN
1,攝影記者,18.0,1000000.0,1000000.0,NaN,300000.0
2,業務經理,21.0,1000000.0,9000000.0,NaN,NaN
3,工程師,2.0,1000000.0,1000000.0,NaN,300000.0
4,高級工程師,2.0,1000000.0,1000000.0,NaN,300000.0


,各業受僱員工(總薪資)(109年7月)(單位：元),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115
0,NaN,工業及服務業,工業,礦業及土石採取業,石油及天然氣礦業,砂、石採取及其他礦業,製造業,食品及飼品製造業,飲料及菸草製造業,紡織業,...,醫療保健及社會工作服務業,醫療保健業,社會工作服務業,藝術、娛樂及休閒服務業,創作及藝術表演業,運動、娛樂及休閒服務業,其他服務業,個人及家庭用品維修業,美髮及美容美體業,其他個人服務業
1,NaN,總薪資,總薪資,總薪資,總薪資,總薪資,總薪資,總薪資,總薪資,總薪資,...,總薪資,總薪資,總薪資,總薪資,總薪資,總薪資,總薪資,總薪資,總薪資,總薪資
2,(888800)總計,"55,042","57,522","106,642","212,658","38,812","57,709","41,661","65,156","36,475",...,"59,250","63,240","30,604","38,317","39,836","38,054","33,925","36,532","29,349","36,372"
3,(100000)主管及監督人員,"105,985","128,276","205,785","388,688","89,800","129,663","82,520","109,598","67,269",...,"114,097","125,193","47,265","75,026","93,607","71,689","58,622","59,818","50,534","64,484"
4,(112000)高階主管(總經理及總執行長),"212,514","261,443","193,432","372,613","85,923","271,264","135,876","151,912","131,955",...,"232,391","250,713","79,755","148,416","225,490","133,831","161,614","148,014","188,286","171,692"


In [213]:
# 各職業平均薪水
raw_salary = gov['各業受僱員工(總薪資)(109年7月)(單位：元)'].values
salary_list = []
for x in raw_salary:
    x = str(x)
    x = re.sub(r'[^\w]', '', x)
    x = re.compile(u'[\u4E00-\u9FA5|\s]').findall(x)
    x = "".join(x)
    if x:
        salary_list.append(x)
    else:
        salary_list.append('無')
        
print(salary_list[:10])

['無', '無', '總計', '主管及監督人員', '高階主管總經理及總執行長', '中階主管經理', '監督人員含現場及辦公室主任科股長', '專業人員', '物理化學及地球科學專業人員', '精算師取得正式資格者']


In [214]:
# 產業
occupation_dict = {
    1: "營造／礦砂業",
    2: "製造業",
    3: "水電燃氣業",
    4: "批發／零售／貿易商",
    5: "旅宿／餐飲",
    6: "運輸倉儲",
    7: "農／林／漁／牧",
    8: "金融保險",
    9: "不動產／租賃",
    10: "軍／警／消",
    11: "公務人員",
    12: "律師／會計師／地政士",
    13: "醫藥服務",
    14: "休閒服務",
    15: "其他／家管",
    16: "自由業",
    17: "投資／自營商",
    18: "媒體文教",
    19: "學生",
    20: "學校教師（含行政人員）",
    21: "資訊科技",
    22: "公證人或記帳士",
    23: "國防工業",
    24: "投資或稅務顧問公司",
    25: "不動產仲介／代銷商",
    26: "大宗物資貿易商",
    27: "博弈業（網路／實體）",
    28: "八大特種行業",
    29: "宗教、慈善、基金會",
    30: "銀樓、珠寶商",
    31: "藝術品或古董買賣商",
    32: "當鋪",
    33: "實體或虛擬貨幣兌換所"
}

In [215]:
job_list = yushan['job_title'].values
job_list[:10]


array(['維護工程師', '攝影記者', '業務經理', '工程師', '高級工程師', '業務', '工程師', '行政人員', '副理',
       '高級工程師'], dtype=object)

In [236]:
def get_features(job_title, statistics=None):
    
    assert statistics is not None
    
    global salary_list
    global gov
    
    if job_title == '無':
        return 0
    if job_title is None:
        return 0
    
    score_list = []
    for x in salary_list[2:]:
        score_list.append(nltk.edit_distance(list(job_title), list(x)))
    idx_list = np.argsort(score_list)
    
    subidx = 0
    while True:
        res = []    
        for x in gov.iloc[idx_list[subidx], :].values:
            if isinstance(x, str):
                x = x.replace(',', '')
                if x.isdigit():
                    res.append(int(x))
        if len(res) > 0:
            if statistics == 'mean':
                return np.mean(res)
            if statistics == 'median':
                return np.median(res)
            if statistics == 'std':
                return np.std(res)
        subidx += 1

In [219]:
# def get_features(job_title):
#     if job_title == '無':
#         return 0
#     if job_title is None:
#         return 0
#     global gov
#     idx = get_closest_idx(job_title)
#     res = []
#     for x in gov.iloc[idx, :].values:
#         if isinstance(x, str):
#             x = x.replace(',', '')
#             if x.isdigit():
#                 res.append(int(x))
#     # print(np.mean(res))
#     # print(np.std(res))
#     # print(np.median(res))
#     if len(res) > 0:
#         return np.median(res)
#     else:
#         print(idx, job_title, end=' / ')
#         return 0

In [241]:
from tqdm import tqdm
salary_median = []
for x in tqdm(yushan['job_title']):
    salary_median.append(get_features(x, 'std')) # choose your statistics here

100%|██████████| 131469/131469 [10:55<00:00, 200.53it/s]


In [242]:
print(len(yushan['job_title']))
print(sum(np.array(salary_median) == 0))

131469
9850


In [243]:
yushan['salary_std'] = salary_median

In [245]:
yushan.to_csv('./data/job_extened.csv', index=None)